# Machine Learning Based Predictor


## First, Lets get the data. Below is one monthly data from MCX

In [74]:


import pandas as pd

data = pd.read_csv('C:\\Users\\kaust\\Downloads\\WinPython-64bit-3.5.2.3Qt5\\myScripts\\MCX_monthlyData.txt')
data = data.drop(['DATE'],axis=1)
data.head()

,OPEN,HIGH,LOW,CLOSE,VOLUME
0,1152.00,1159.00,1152.00,1157.90,4977
1,1157.00,1158.70,1156.00,1156.00,2765
2,1156.00,1156.00,1150.40,1154.15,2160
3,1154.15,1154.35,1153.05,1153.90,780
4,1153.80,1155.00,1153.20,1154.05,1390


# Here comes the most interesting part. What we want to build is predictor which will take in as a input the few moving avarages.
# The goal of the predictor is to decide when the position can be entered so that the position returns the profit during particular period. 

Below are moving avarages and return window we are using.

1. Moving Avarage 1 = 400 minutes
2. Moving Avarage 2 = 120 minutes
3. Moving Avarage 3 = 20 minutes
4. The time during which position will return profit
5. How much minimum profit it should return during above time.

In [75]:

last = 9999999
MovingHigh = []
MovingLow = []
MovingAvarage = []
MovingAvarage_2 = []
MovingAvarage_3 = []
window = 400
window_2 = 20
window_3 = 120
windowEnter = 15
EnterDifference = 13
Enter = []
# Up = 1 down = 0
for index, row in data.iterrows():
   
    
    if (index+windowEnter <= data.shape[0]):
        EnterDiff = [x - row['OPEN'] for x in data['OPEN'][index+1:index+windowEnter]] 
        if(max(EnterDiff)>=EnterDifference):
            Enter.append(1)
        else:
            Enter.append(0)
    else:
        Enter.append(0)
            
    if(index < window):
        MovingAvarage.append(99999999)
    else:
        MovingAvarage.append(data['OPEN'][(index-window):index].mean())
        
    if(index < window_3):
        MovingAvarage_3.append(99999999)
    else:
        MovingAvarage_3.append(data['OPEN'][(index-window_3):index].mean())
        
    if(index < window_2):
        MovingAvarage_2.append(99999999)
        MovingHigh.append(99999999)
        MovingLow.append(99999999)
    else:
        MovingHigh.append(data['HIGH'][(index-window):index].max())
        MovingLow.append(data['LOW'][(index-window):index].max())
        MovingAvarage_2.append(data['OPEN'][(index-window_2):index].mean())
        
data['Enter'] = Enter
data['MovingHigh'] = MovingHigh
data['MovingAvarage'] = MovingAvarage
data['MovingAvarage_2'] = MovingAvarage_2
data['MovingAvarage_3'] = MovingAvarage_3
data['MovingLow'] = MovingLow
data.head()

,OPEN,HIGH,LOW,CLOSE,VOLUME,Enter,MovingHigh,MovingAvarage,MovingAvarage_2,MovingAvarage_3,MovingLow
0,1152.00,1159.00,1152.00,1157.90,4977,0,99999999.0,99999999.0,99999999.0,99999999.0,99999999.0
1,1157.00,1158.70,1156.00,1156.00,2765,0,99999999.0,99999999.0,99999999.0,99999999.0,99999999.0
2,1156.00,1156.00,1150.40,1154.15,2160,0,99999999.0,99999999.0,99999999.0,99999999.0,99999999.0
3,1154.15,1154.35,1153.05,1153.90,780,0,99999999.0,99999999.0,99999999.0,99999999.0,99999999.0
4,1153.80,1155.00,1153.20,1154.05,1390,0,99999999.0,99999999.0,99999999.0,99999999.0,99999999.0


## So basically , if u see above we have change the prediction to Binary value. 1 for correct position and 0 for wrong position.

## Lets now build the predictor. 

In [76]:
%matplotlib inline

import matplotlib.pyplot as plt



In [77]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
#import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [78]:
data_copy = data[window:]
data_copy = data_copy.drop(['CLOSE','HIGH','LOW'],axis=1)
data_copy.head()


,OPEN,VOLUME,Enter,MovingHigh,MovingAvarage,MovingAvarage_2,MovingAvarage_3,MovingLow
400,1124.45,1849,0,1159.35,1143.844375,1128.5050,1129.505417,1157.0
401,1123.50,618,0,1159.35,1143.775500,1127.9525,1129.475833,1157.0
402,1123.00,403,0,1159.35,1143.691750,1127.4525,1129.437500,1157.0
403,1123.40,1895,0,1159.35,1143.609250,1126.9025,1129.405000,1157.0
404,1125.25,1246,0,1159.35,1143.532375,1126.3775,1129.366667,1157.0


In [79]:
y = data_copy['Enter'].values
data_copy = data_copy.drop(['Enter'],axis=1)
X = data_copy.values
data_copy.head()

,OPEN,VOLUME,MovingHigh,MovingAvarage,MovingAvarage_2,MovingAvarage_3,MovingLow
400,1124.45,1849,1159.35,1143.844375,1128.5050,1129.505417,1157.0
401,1123.50,618,1159.35,1143.775500,1127.9525,1129.475833,1157.0
402,1123.00,403,1159.35,1143.691750,1127.4525,1129.437500,1157.0
403,1123.40,1895,1159.35,1143.609250,1126.9025,1129.405000,1157.0
404,1125.25,1246,1159.35,1143.532375,1126.3775,1129.366667,1157.0


## Below is training mode of the predictor. Here predictor decides how to use below parameters 
1. OPEN
2. VOLUME
3. Moving Avarage 1
4. Moving Avarage 2
5. Moving Avarage 3


In [80]:
#Xtrain, Xtest = X[:int(len(X) * 0.50)], X[int(len(X) * 0.50):] 
#ytrain, ytest = y[:int(len(y) * 0.50)], y[int(len(y) * 0.50):] 
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.50)
print(Xtrain.shape)
print(Xtest.shape)


(3925, 7)
(3925, 7)


## Now its time to test our predictor.

In [85]:


gadaboost = GradientBoostingClassifier()
gadaboost.fit(Xtrain, ytrain)
y_val_l = gadaboost.predict_proba(Xtest)
print("Validation accuracy: ", sum(pd.DataFrame(y_val_l).idxmax(axis=1).values
                                   == ytest)/len(ytest))



Validation accuracy:  0.992356687898


## Dont get too excited, at the end we have to check the false positive and false nagative range

In [86]:
import pickle
with open('C:\\Users\\kaust\\Downloads\\WinPython-64bit-3.5.2.3Qt5\\myScripts\\my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(gadaboost, fid)    


In [87]:
target = open('C:\\Users\\kaust\\Downloads\\WinPython-64bit-3.5.2.3Qt5\\myScripts\\out.txt', 'w')
y_pre = pd.DataFrame(y_val_l).idxmax(axis=1).values

Total_Trades = 0
Total_Matched_Trades = 0
Total_False_Positives = 0
Total_False_Negatives = 0

for i in range(Xtest.shape[0]):
    line = str(ytest[i])+','+str(y_pre[i])
    target.write(line)
    target.write('\n')
    if(ytest[i] == 1):
       Total_Trades = Total_Trades + 1
    if(ytest[i] == 1 and y_pre[i] == 1):
        Total_Matched_Trades = Total_Matched_Trades + 1
    if(ytest[i] == 0 and y_pre[i] == 1):
        Total_False_Positives = Total_False_Positives + 1
    if(ytest[i] == 1 and y_pre[i] == 0):
        Total_False_Negatives = Total_False_Negatives + 1
target.close()

print("Total Trades = "+str(Total_Trades))
print("Total Matched Trades = "+str(Total_Matched_Trades))
print("Total False Positive Trades = "+str(Total_False_Positives))
print("Total False Negative Trades = "+str(Total_False_Negatives))


Total Trades = 44
Total Matched Trades = 22
Total False Positive Trades = 8
Total False Negative Trades = 22


 ## So , as you can see that this predictor is not good at all, it gives 8 false positive and whoping 22 false negatives.

In [88]:
import numpy as np
indices = np.argsort(gadaboost.feature_importances_)[::-1]

# Print the feature ranking
print('Feature ranking:')

for f in range(data_copy.shape[1]):
    print('%d. feature %d %s (%f)' % (f+1 , indices[f], data_copy.columns[indices[f]],
                                      gadaboost.feature_importances_[indices[f]]))

Feature ranking:
1. feature 1 VOLUME (0.303253)
2. feature 5 MovingAvarage_3 (0.211591)
3. feature 4 MovingAvarage_2 (0.211256)
4. feature 3 MovingAvarage (0.134678)
5. feature 0 OPEN (0.076271)
6. feature 2 MovingHigh (0.037919)
7. feature 6 MovingLow (0.025033)


## Summary - Its just naive attempt to show one of the ways in which ML can be used to build the predictor.  